## Example -  setting up the Neo4j Driver

In [ ]:
url = "bolt://localhost:7687"
user = "neo4j"
password = "sunshine"

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import pprint

class NeoDriver(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Created Neo4j driver. URI=" + uri)

    def close(self):
        self._driver.close()

    def query(self, query, **kwargs):
        with self._driver.session() as session:
            return session.write_transaction(self.run_cypher, query, **kwargs)

    @staticmethod
    def run_cypher(tx, query, **kwargs):
        return tx.run(query, **kwargs)
    
driver = NeoDriver(url, user, password)

## Example - Reading Neo4j results

In [ ]:
# Run a Cypher query and display the results.
result = driver.query("""
MATCH (p:Product) 
RETURN p LIMIT 5
""")

for row in result.data():
    print(row['p']['partNumber'] + ": " + row['p']['shortDescription'])

## Example algorithm - similarity between departments
### Step 1 - estimate required size

In [ ]:
# Create a Cypher projection graph of similar departments (based on shared products) 
result = driver.query("""
CALL gds.graph.create.cypher.estimate(
    'MATCH (d:Department) RETURN id(d) as id',
    'MATCH (d:Department)<-[:HAS_DEPARTMENT]->(p:Product)-[:HAS_DEPARTMENT]->(d2:Department) RETURN id(d) AS source, id(d2) AS target')
""")

for row in result.data():
    print(row['requiredMemory'] + " memory required")
    print(str(row['nodeCount']) + " nodes expected")
    print(str(row['relationshipCount']) + " rels expected")

### Step 2 - clear existing in-memory graph

In [ ]:
# This query drops the graph if it already exists, else it does nothing.
result = driver.query("""
CALL gds.graph.exists($name) YIELD exists
WHERE exists
CALL gds.graph.drop($name) yield graphName
RETURN *
""", name = 'departments-products')

print(result.data())


### Step 3: create new graph projection

In [ ]:
# Create a Cypher projection graph of similar departments (based on shared products) 
result = driver.query("""
CALL gds.graph.create.cypher(
    'departments-products',
    'MATCH (d:Department) RETURN id(d) as id',
    'MATCH (d:Department)<-[:HAS_DEPARTMENT]->(p:Product)-[:HAS_DEPARTMENT]->(d2:Department) RETURN id(d) AS source, id(d2) AS target')
""")

for row in result.data():
    print(str(row['nodeCount']) + " nodes projected")
    print(str(row['relationshipCount']) + " rels projected")
    print(str(row['createMillis']) + " ms to create")

### Step 4: estimate algorithm memory

In [ ]:
result = driver.query("""
CALL gds.nodeSimilarity.stream.estimate('departments-products',  { similarityCutoff: 0.5 })
""")

print(result.data()[0]['requiredMemory'] + ' memory required')

### Step 5 - run the algorithm (stream mode)


In [ ]:
result = driver.query("""
CALL gds.nodeSimilarity.stream('departments-products', { similarityCutoff: 0.75 })
""")

df = pd.DataFrame(result.data())
print(df)


### Step 5 - run the algorithm (stats mode)

In [ ]:
result = driver.query("""
CALL gds.nodeSimilarity.stats('departments-products', { similarityCutoff: 0.75 })
""")

pprint.pprint(result.data())

### Step 5 - run the algorithm (write mode)

In [ ]:
driver.query("""
MATCH (:Department)-[s:SIMILAR_TO]->(:Department)
DELETE s
""")

result = driver.query("""
CALL gds.nodeSimilarity.write('departments-products', { writeProperty: 'similarity', writeRelationshipType: 'SIMILAR_TO', similarityCutoff: 0.75})
""")

pprint.pprint(result.data())